## 과제 Book 클래스

In [56]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __repr__(self):
        return '<Person : {}, {}>'.format(self.name, self.age)
    
    def __str__(self):
        return self.name

people = [
    Person('Tom', 10),
    Person('Stve', 12),
    Person('John', 8),
]

In [54]:
people

In [55]:
print(people[0])  # print(str(people[0]))

Tom


In [2]:
first_people = people[0]

In [3]:
first_people.name

'Tom'

In [4]:
first_people.age

10

In [5]:
getattr(first_people, 'name')

'Tom'

In [6]:
getattr(first_people, 'age')

10

In [9]:
# getattr
print(first_people.name)
print(getattr(first_people, 'name'))

# setattr
# first_people.age = 20
setattr(first_people, 'age', 20)
print(first_people.age)

# hasattr
print(hasattr(first_people, 'age'))
print(hasattr(first_people, 'region'))

Tom
Tom
20
True
False


In [10]:
def sortfn(list, standard_name):
    return sorted(list, key = lambda i: getattr(i, standard_name))

In [13]:
for person in people:
    print(person.name, person.age)

Tom 20
Stve 12
John 8


In [15]:
for person in sortfn(people, 'age'):
    print(person.name, person.age)

John 8
Stve 12
Tom 20


## 장식자 base

In [23]:
def base_10(fn):
    def wrap(x, y):
        print("called base_10")
        return fn(x, y) + 10
    return wrap

def base_20(fn):
    def wrap(x, y):
        print("called base_20")
        return fn(x, y) + 20
    return wrap

def base_30(fn):
    def wrap(x, y):
        print("called base_30")
        return fn(x, y) + 30
    return wrap

@base_10
@base_20
@base_30
def mysum(x, y):
    return x + y

# mysum = base_30(mysum)
# mysum = base_20(mysum)
# mysum = base_10(mysum)

print(mysum(1, 2))

called base_10
called base_20
called base_30
63


### 가변인자를 처리하는 장식자

In [24]:
def base_10(fn):
    def wrap(*args):
        return fn(*args) + 10
    return wrap

@base_10
def mysum2(x, y):
    return x + y

@base_10
def mysum3(x, y, z):
    return x + y + z

print(mysum2(1, 2))
print(mysum3(1, 2, 3))

13
16


### 인자를 받는 장식자

In [26]:
def base(i):
    def wrap(fn):
        def inner(*args):
            return fn(*args) + i
        return inner
    return wrap

@base(10)
def mysum2(x, y):
    return x + y

print(mysum2(1, 2))

13


In [30]:
def myfilter(filter_fn):
    def wrap(fn):
        def inner(*args):
            # args  # (1, 2, 3, 4) => (0, 2, 0, 4)
            
            filtered_args = (i if filter_fn(i) else 0 for i in args)

            '''
            filtered_args = []
            for i in args:
                if filter_fn(i):
                    filtered_args.append(i)
                else:
                    filtered_args.append(0)
            '''
            return fn(*filtered_args)
        return inner
    return wrap

@myfilter(lambda i: i%2==0)
def mysum4(a, b, c, d):
    return a + b + c + d

print(mysum4(1, 2, 3, 4))

6


### 클래스로 장식자 만들기

In [34]:
class Base:
    def __init__(self, i):
        self.i = i
    
    def __call__(self, fn):
        def wrap(*args):
            return self.process(fn(*args))
        return wrap
    
    def process(self, value):
        return value + self.i

class Multiply(Base):
    def process(self, value):
        return value * self.i
    
# @Base(10)
@Multiply(10)
def mysum2(x, y):
    return x + y

# base_10_instance = Base(10)
# mysum2 = base_10_instance(mysum2)

print(mysum2(1, 2))

30


In [38]:
def myfilter(filter_fn):
    def wrap(fn):
        def inner(*args):
            filtered_args = (i if filter_fn(i) else 0 for i in args)
            return fn(*filtered_args)
        return inner
    return wrap


class MyFilter:
    def __init__(self, filter_fn):
        self.filter_fn = filter_fn
    
    def __call__(self, fn):
        def wrap(*args):
            filtered_args = (i if self.filter_fn(i) else 0 for i in args)
            return fn(*filtered_args)
        return wrap
    

# @myfilter(lambda i: i%2==0)
@MyFilter(lambda i: i%2==0)
def mysum4(a, b, c, d):
    return a + b + c + d

print(mysum4(1, 2, 3, 4))

6


### 클래스를 장식하기

In [44]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def say_hello(self):
        print('Hello, {}.'.format(self.name))

Person.byebye = lambda self: print('bye bye {}'.format(self.name))
Person.say_hello = lambda self: print('Hi Hi {}'.format(self.name))
        
tom = Person('Tom', 10)
tom.region = 'Seoul'
tom.say_hello()
tom.byebye()

Hi Hi Tom
bye bye Tom


In [ ]:

import sys

PY2 = sys.version_info[0] == 2

def python_2_unicode_compatible(klass):
    if PY2:
        klass.__unicode__ = klass.__str__
        klass.__str__ = lambda self: self.__unicode__().encode('utf-8')
    return klass

@python_2_unicode_compatible
class Person:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name


In [61]:
def base_klass(i):
    def wrap(klass):
        klass.BASE = i
        return klass
    return wrap

@base_klass(100)
class Person:
    BASE = 0

    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def old(self, years):
        self.age += (self.BASE + years)

tom = Person('Tom', 8)
tom.old(10)
print(tom.age)

118


In [57]:
import sys
sys.version_info

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)

In [58]:
sys.version_info[:3]

(3, 5, 2)

In [59]:
sys.version_info[0]

3